# Stream test  
hdfs 에 구매 요청 정보가 들어오는 경우, stream 방식으로 요청을 수신하고 처리  

In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
APP_NAME = 'stream-kafka'


# 스파크 생성 
def spark_creation_yarn():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '4g')\
    .config('spark.num.executors', '2')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '4g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .getOrCreate()
    sc = spark.sparkContext
    return spark

def spark_creation_local():
    kafka_spark_jar_path = "/spark-git/work-k/jar/spark-streaming-kafka-0-10_2.12-3.2.1.jar,/spark-git/work-k/jar/spark-sql-kafka-0-10_2.12-3.2.1.jar,/spark-git/work-k/jar/kafka-clients-0.10.0.1.jar"
    spark = SparkSession.builder.master('local[*]').appName(APP_NAME)\
    .config("spark.jars", kafka_spark_jar_path)\
    .config("spark.driver.extraClassPath", kafka_spark_jar_path)\
    .getOrCreate()
# .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12-3.2.1")\
    #    .config("spark.driver.extraClassPath", kafka_spark_jar_path)\
    #     .config("spark.jars.packages", pydeequ.deequ_maven_coord) \
    # .config("spark.jars.excludes", pydeequ.f2j_maven_coord) \
    sc = spark.sparkContext
    # spark.conf.set("spark.driver.extraClassPath", kafka_spark_jar_path)
    return spark
def spark_creation():
    return spark_creation_local()

def display_output(outTarget, sqlString, interval = 1):
    from IPython.display import display, clear_output
    from pyspark.sql.streaming import StreamingQuery 
    import time 
    if isinstance(outTarget, StreamingQuery):
        while True:
            clear_output(wait=False)
            print(f"Query : {sqlString}")
            # display(spark.sql(sqlString).show())
            print(f"outTarget : {outTarget}")
            # print(f"temp : {(temp[0])}")
            #display(outTarget.select("type").show())
            time.sleep(interval)
    else:
        print("Not instance......")
        spark.sql(sqlString).show()
        
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType
    columns = [
        StructField("type", StringType())
        , StructField("qty", LongType())
    ]
    inven_schema = StructType(columns)
    return inven_schema

In [2]:
%%time
spark = spark_creation()

spark

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


CPU times: user 23.8 ms, sys: 46 ms, total: 69.9 ms
Wall time: 6.47 s


In [3]:
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "quickstart-events") \
  .load()

# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
# df.createOrReplaceTempView("mytable")
query_name = "lines_12"
# lines.createOrReplaceTempView(query_name)
sql = f"select * from mytable "


In [5]:
# activityQuery = df.writeStream.queryName(query_name).format("memory").outputMode("append").start()
# activityQuery.awaitTermination() 
# spark.streams.active

# from time import sleep
# for x in range(3):
#     spark.sql(sql).show(3)
#     sleep(2)

In [5]:
outQ = df \
    .withWatermark("timestamp", "5 seconds")\
    .writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("mytable") \
    .start()

outQ.awaitTermination()


StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: mytable [id = 40a1c68e-3baa-44e6-baed-20d601545080, runId = 6b308e1d-b9df-4fd1-9ab6-63bc50384942]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [ ]:
from IPython.display import display, clear_output
import time  

while True:
    clear_output(wait=True)
    display(outQ.status)
    display(outQ.lastProgress)
    display(spark.sql(f'SELECT * FROM mytable').show())
    time.sleep(3)

In [11]:
outQ.status

{'message': 'Terminated with exception: org/apache/spark/kafka010/KafkaConfigUpdater',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [5]:
spark.stop()